In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# reduce memory usage

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
def read_data():
    
    print('Reading files...')
    
    calendar = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    
    sell_prices = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    
    sales_train_validation = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    
    sales_train_evaluation = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
    print('Sales train evaluation has {} rows and {} columns'.format(sales_train_evaluation.shape[0], sales_train_evaluation.shape[1]))
    
    submission = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv')
    
    return calendar, sell_prices, sales_train_validation, sales_train_evaluation, submission

calendar, sell_prices, sales_train_validation, sales_train_evaluation, submission = read_data()

In [ ]:
# make train data

df = sales_train_evaluation
df = df.loc[:, "d_1817":]     # drop unneeded columns
df.insert(0, 'id', sales_train_evaluation["id"])     # insert needed columns
df.insert(1, 'item_id', sales_train_evaluation["item_id"])
df.insert(2, 'dept_id', sales_train_evaluation["dept_id"])
df.insert(3, 'cat_id', sales_train_evaluation["cat_id"])
df.insert(4, 'store_id', sales_train_evaluation["store_id"])
df.insert(5, 'state_id', sales_train_evaluation["state_id"])
df

In [ ]:
# melt and merge training data

df = pd.melt(df, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'actual_value')
df = pd.merge(df, calendar, how="left", left_on="day", right_on="d")
df = pd.merge(df, sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
df

In [ ]:
# insert custom features

def simple_fe(data):
    
    # rolling demand features
    data['rolling_mean_t7'] = data.groupby(['id'])['actual_value'].transform(lambda x: x.shift(28).rolling(7).mean())
    data['rolling_std_t7'] = data.groupby(['id'])['actual_value'].transform(lambda x: x.shift(28).rolling(7).std())
    data['rolling_mean_t30'] = data.groupby(['id'])['actual_value'].transform(lambda x: x.shift(28).rolling(30).mean())
    data['rolling_std_t30'] = data.groupby(['id'])['actual_value'].transform(lambda x: x.shift(28).rolling(30).std())
    
    
    # price features
    data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
    data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1'])
    data['rolling_price_std_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
    data['rolling_price_std_t30'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())

    return data

In [ ]:
# drop unneeded columns

df = simple_fe(df)
df = df.truncate(before=2957530)
df['id'] = df['id'].str.replace('_evaluation','_validation')
id_list = df["id"]     # get id list
df

In [ ]:
# make predict data

lstm_csv = pd.read_csv('/kaggle/input/lstm-init/lstm_pred.csv')
lstm_csv = lstm_csv.loc[:, "id":]
lstm_csv.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 
                     'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                      'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
lstm_csv

In [ ]:
test = pd.melt(lstm_csv, id_vars = ['id'], var_name = 'day', value_name = 'lstm_predicted')
df = df.reset_index()
df = pd.merge(df, test, on=['id', 'day'] )

df = df.drop(["id", 'index', 'date', 'wm_yr_wk', 'weekday', 'd', 'day'], axis=1)
df

In [ ]:
# reduce mem usage, fill NaN, categoical data tranforms

from sklearn import preprocessing

nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in nan_features:
    df[feature].fillna('unknown', inplace = True)

cat = ['item_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']

for feature in cat:
    encoder = preprocessing.LabelEncoder()
    df[feature] = encoder.fit_transform(df[feature])

dummies = ['dept_id', 'cat_id', 'store_id', 'state_id', 'wday', 'month', 'year']
df = pd.get_dummies(df, columns = dummies)

df = reduce_mem_usage(df)
df.info()

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(["actual_value"], axis=1)
y = df["actual_value"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

In [ ]:
import lightgbm as lgb

params = {
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective': 'regression',
        'n_jobs': -1,
        'seed': 236,
        'learning_rate': 0.5,
        'bagging_fraction': 0.8,
        'bagging_freq': 10 ,
        'colsample_bytree': 0.8
}

train_set = lgb.Dataset(X_train, y_train)
val_set = lgb.Dataset(X_test, y_test)

model = lgb.train(params, train_set, num_boost_round = 2500, early_stopping_rounds = 100, valid_sets = [train_set, val_set], verbose_eval = 100)

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error

predictions = model.predict(X_train)

print(predictions)

print(y_train)

print("MSE: ", mean_squared_error(y_train, predictions) )
print("RMSE: ", sqrt(mean_squared_error(y_train, predictions)) )

In [ ]:
# plot decision tree

import matplotlib.pyplot as plt

lgb.plot_tree(model)
fig = plt.gcf()
fig.set_size_inches(25, 17)

plt.savefig('foo.png')

In [ ]:
plt.figure(figsize=(20, 20))
lgb.plot_importance(model)
plt.title("Featurertances")
plt.show()

In [ ]:
predictions = model.predict(X)

predictions = np.reshape(predictions, (30490, 28))

print(predictions)

In [ ]:
final = pd.DataFrame()

final["id"] = lstm_csv["id"]
for i in range(28):
    final['F' + str(i + 1)] = ""

for i in range(30490):
    
    if i % 1000 == 0:
        print(i)
    
    final.loc[ i , 1:] =predictions[i]

final

In [ ]:
final.to_csv("final.csv")